#### Imports

In [ ]:
from typing import Union
from typing_extensions import NotRequired, TypedDict
from functools import reduce

from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import numpy as np
import xarray as xr


from sdm_eurec4a.visulization import (
    adjust_lightness_array,
    set_custom_rcParams,
    handler_map_alpha,
)
from sdm_eurec4a.reductions import mean_and_stderror_of_mean

from sdm_eurec4a import RepositoryPath
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def set_xticks_time(ax):
    xticks = [0, 500, 1000]
    ax.set_xticks(xticks)


def set_yticks_height(ax):
    yticks = [0, 500, 1000, 1500, 2000]
    ax.set_yticks(yticks)


def set_yticks_height_km(ax):
    yticks = [0, 0.5, 1, 1.5, 2]
    ax.set_yticks(yticks)


def set_logxticks_meter(ax):
    xticks = [1e-6, 1e-3]
    xticklabels = [r"$10^{-6}$", r"$10^{-3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logxticks_micrometer(ax):
    xticks = [1e-3, 1e0, 1e3]
    xticklabels = [r"$10^{-3}$", r"$10^{0}$", r"$10^{3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logtyticks_psd(ax):
    yticks = [1e0, 1e6]
    yticklabels = [r"$10^0$", r"$10^6$"]
    ax.set_yticks(yticks, yticklabels)


def set_yticks_lwc(ax):
    ax.set_yticks([0, 0.1, 0.2])

In [ ]:
def label_from_attrs(da: xr.DataArray, return_name=True, return_units=True):
    try:
        name = f"{da.attrs['long_name']}"
    except KeyError:
        name = f"{da.name}"

    if "units" in da.attrs:
        units = f"{da.attrs['units']}"
        if "$" not in units:
            units = f"${units}$"

        units = units.replace("$", " ")
        units = rf"$\left[ {units} \right]$"
    else:
        units = "[???]"

    if return_name == True and return_units == True:
        return f"{name} {units}"
    elif return_name == True and return_units == False:
        return f"{name}"
    elif return_name == False and return_units == True:
        return f"{units}"
    else:
        return ""

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
plt.rcParams.update(
    {
        "axes.spines.top": False,
        "axes.spines.right": False,
        "axes.spines.left": False,
        "axes.spines.bottom": False,
    }
)

dark_colors = adjust_lightness_array(default_colors, amount=0.5)

repo_path = RepositoryPath("levante")()
print(repo_path)

subdata_dir = "output_v3.5"
data_path = Path("/home/m/m301096/CLEO/data/") / subdata_dir

ds_subpath = "combined/eulerian_dataset_combined_v2.nc"

# THE PATH TO THE SCRIPT DIRECTORY
script_dir = Path("/home/m/m301096/repositories/sdm-eurec4a/notebooks/thesis/results/")
print(script_dir)


fig_dir = repo_path / "results" / script_dir.relative_to(repo_path) / subdata_dir
print(fig_dir)

fig_dir.mkdir(parents=True, exist_ok=True)


microphysics = (
    "null_microphysics",
    "condensation",
    "collision_condensation",
    "coalbure_condensation_large",
)

/home/m/m301096/repositories/sdm-eurec4a
/home/m/m301096/repositories/sdm-eurec4a/notebooks/thesis/results
/home/m/m301096/repositories/sdm-eurec4a/results/notebooks/thesis/results/output_v3.5


# Set time slice to use for temporal mean and median

In [ ]:
time_slice = slice(1500, 3590)  # seconds
radius_split = 45  # µm
radius_slice = slice(1e0, None)  # µm

# Load datasets

In [ ]:
class MicrophysicDict(TypedDict):
    dataset: xr.Dataset
    microphysics: str
    path: Path
    # linestyle: Union[str, tuple]
    # color: str


class OptionalDictOfMicrophysicDict(TypedDict):
    null_microphysics: NotRequired[MicrophysicDict]
    condensation: NotRequired[MicrophysicDict]
    collision_condensation: NotRequired[MicrophysicDict]
    coalbure_condensation_large: NotRequired[MicrophysicDict]
    coalbure_condensation_small: NotRequired[MicrophysicDict]


class DictOfMicrophysicDict(TypedDict):
    null_microphysics: MicrophysicDict
    condensation: MicrophysicDict
    collision_condensation: MicrophysicDict
    coalbure_condensation_cke: MicrophysicDict
    coalbure_condensation_large: MicrophysicDict
    coalbure_condensation_small: MicrophysicDict


data_dict = DictOfMicrophysicDict(
    null_microphysics=MicrophysicDict(
        microphysics="Null microphysics",
        path=Path(),
        dataset=xr.Dataset(),
    ),
    condensation=MicrophysicDict(
        microphysics="Condensation/Evaporation",
        path=Path(),
        dataset=xr.Dataset(),
    ),
    collision_condensation=MicrophysicDict(
        microphysics="Coll-coal, cond./evap.",
        path=Path(),
        dataset=xr.Dataset(),
    ),
    coalbure_condensation_cke=MicrophysicDict(
        microphysics="Coll-coal-breakup n by CKE\nand cond./evap.",
        path=Path(),
        dataset=xr.Dataset(),
    ),
    coalbure_condensation_large=MicrophysicDict(
        microphysics="Coll-coal-breakup (n=125)\nand cond./evap.",
        path=Path(),
        dataset=xr.Dataset(),
    ),
    coalbure_condensation_small=MicrophysicDict(
        microphysics="Coll-coal-breakup (n=5)\nand cond./evap.",
        path=Path(),
        dataset=xr.Dataset(),
    ),
)

for mp in data_dict:
    data_dict[mp]["path"] = data_path / f"{mp}" / ds_subpath

for key in data_dict:
    ds = xr.open_dataset(data_dict[key]["path"], chunks={"cloud_id": 2})
    # ds = ds.sel(cloud_id = [18, 301])
    ds.attrs.update(microphysics=data_dict[key]["microphysics"])
    ds.attrs.update(microphysics_short=key)

    data_dict[key]["dataset"] = ds

# ---------------------------------------------------- #
# Reindex the datasets to have the same radius bins
# ---------------------------------------------------- #

combined_radius_bins = reduce(
    np.union1d, [data_dict[mp]["dataset"]["radius_bins"].values for mp in data_dict]
)
fill_value = np.nan
print("Number of radius bins:", len(combined_radius_bins))
print("Fill value:", fill_value)
for mp in data_dict:
    data_dict[mp]["dataset"] = data_dict[mp]["dataset"].reindex(
        radius_bins=combined_radius_bins, fill_value=fill_value
    )

Number of radius bins: 76
Fill value: nan


### All clouds which are simulated

In [ ]:
intersect_cloud_ids = reduce(
    np.intersect1d, [data_dict[key]["dataset"]["cloud_id"].data for key in data_dict]
)
for mp in data_dict:
    print(mp, len(data_dict[mp]["dataset"]["cloud_id"]))

print("\nIntersect:", len(intersect_cloud_ids))
print("cloud_ids:", intersect_cloud_ids)

null_microphysics 94
condensation 78
collision_condensation 83
coalbure_condensation_cke 91
coalbure_condensation_large 93
coalbure_condensation_small 94

Intersect: 66
cloud_ids: [  9  11  18  20  21  22  65  67  68  71  72  73  74  88  94 110 113 114
 130 135 136 142 194 197 198 199 201 203 205 207 208 211 212 213 214 215
 217 218 219 220 221 222 223 224 230 233 235 236 237 292 293 295 296 301
 303 305 306 307 308 309 311 312 314 359 361 362]


In [ ]:
clouds_dict = {
    "222": dict(
        cloud_id=222,
        color="r",
    ),
    "142": dict(
        cloud_id=142,
        color="b",
    ),
}
for key in data_dict:
    ds = data_dict[key]["dataset"]
    for cloud_id in clouds_dict:
        cloud_id = clouds_dict[cloud_id]["cloud_id"]
        is_in = cloud_id in ds["cloud_id"]
        print(f"{cloud_id}, {key}, {is_in}")

222, null_microphysics, True
142, null_microphysics, True
222, condensation, True
142, condensation, True
222, collision_condensation, True
142, collision_condensation, True
222, coalbure_condensation_cke, True
142, coalbure_condensation_cke, True
222, coalbure_condensation_large, True
142, coalbure_condensation_large, True
222, coalbure_condensation_small, True
142, coalbure_condensation_small, True


### Add further variables e.g. latent cooling

In [ ]:
# NOTE: For now, it needs to be divided by 2 s to get values per second Bue to a bug.
def add_variables(
    ds: xr.Dataset, latent_heat_of_condensation: float = 2.265e6, time_slice=time_slice  # J kg-1
):
    # fix attributes
    ds["mass_represented"].attrs["long_name"] = "Mass"

    ds["radius_bins"].attrs["long_name"] = "Radius"
    ds["radius_bins"].attrs["units"] = "$\\mu m$"

    ds["relative_humidity"].attrs["long_name"] = "Relative humidity"
    ds["relative_humidity"].attrs["units"] = "$\\%$"

    ds["gridbox_thickness"] = ds["gridbox_top"] - ds["gridbox_bottom"]
    ds["cloud_altitude"] = ds["gridbox_coord3"].sel(gridbox=ds["max_gridbox"])

    # It seems that xi was stored as an integer. This is not wanted, because nan values will just be large integers.
    ds["xi"] = ds["xi"].astype(float)
    ds["xi"] = ds["xi"].where(ds["xi"] < 1e12)

    ds["xi"].attrs["units"] = "$\\#$"
    ds["xi"].attrs["long_name"] = "Real droplet num. conc."

    ds["xi_per_volume"] = ds["xi"] / ds["gridbox_volume"]
    ds["xi_per_volume"].attrs["long_name"] = ds["xi"].attrs["long_name"]
    ds["xi_per_volume"].attrs["units"] = "$\\# m^{-3}$"

    ds["number_superdroplets_per_volume"] = 1000 * ds["number_superdroplets"] / ds["gridbox_volume"]
    ds["number_superdroplets_per_volume"].attrs["units"] = "$10^3m^{-3}$"
    ds["number_superdroplets_per_volume"].attrs["long_name"] = "Number of superdroplets per volume"

    ds["mass_represented_per_volume"] = 1e3 * ds["mass_represented"] / ds["gridbox_volume"]
    ds["mass_represented_per_volume"].attrs["units"] = "$g m^{-3}$"
    ds["mass_represented_per_volume"].attrs["long_name"] = "Mass"

    ds["evaporation_full"] = 1e3 * ds["massdelta_condensation"].sel(time=time_slice).where(
        ds["sub_cloud_layer_mask"]
    )
    ds["evaporation_full"].attrs["units"] = "$g m^{-3} s^{-1}$"
    ds["evaporation_full"].attrs["long_name"] = "Evaporation rate"
    ds["evaporation_full"].attrs["description"] = "Evaporation rate for all timesteps"

    # ds["evaporation"] = (
    #     ds["evaporation_full"].sel(time=time_slice).mean("time", keep_attrs=True, skipna=True)
    # )
    # ds["evaporation"].attrs["units"] = "$g m^{-3} s^{-1}$"
    # ds["evaporation"].attrs["long_name"] = "Evaporation rate"

    # ds["evaporation"].attrs["units"] = "$g m^{-3} s^{-1}$"
    # ds["evaporation"].attrs["long_name"] = "Evaporation rate"

    ds["latent_heating"] = (
        1e-3
        * ds["evaporation_full"].sel(time=time_slice).mean("time", keep_attrs=True, skipna=True)
        * latent_heat_of_condensation
    )  # kg m-3 s-1 * J kg-1 = W m-3
    ds["latent_heating"].attrs["units"] = "$W m^{-3}$"
    ds["latent_heating"].attrs["long_name"] = "Latent heating"

    ds["latent_heating_full"] = (
        1e-3 * ds["evaporation_full"].sel(time=time_slice) * latent_heat_of_condensation
    )  # kg m-3 s-1 * J kg-1 = W m-3
    ds["latent_heating_full"].attrs["units"] = "$W m^{-3}$"
    ds["latent_heating_full"].attrs["long_name"] = "Latent heating"
    ds["latent_heating_full"].attrs["description"] = "Latent heating for all timesteps"

    ds["latent_heating_mean"] = ds["latent_heating"].mean("gridbox", keep_attrs=True)
    ds["latent_heating_sum"] = (ds["latent_heating"] * ds["gridbox_thickness"]).sum(
        "gridbox", keep_attrs=True
    )
    ds["latent_heating_sum"].attrs["units"] = "$W m^{-2}$"
    ds["latent_heating_sum"].attrs["long_name"] = "Column int. latent heating"

    ds["latent_heating_radius_bins"] = (
        ds["mass_difference_per_volume"].sel(time=time_slice).where(ds["sub_cloud_layer_mask"])
        * latent_heat_of_condensation
    )
    ds["latent_heating_radius_bins"].attrs["units"] = "$W m^{-3}$"
    ds["latent_heating_radius_bins"].attrs["long_name"] = "Latent heating"

    # In order to mask out outliers in the latent heating field,
    # the total latent heating at a timestep for a gridbox should not exceed
    # the minimum or maximum value of the latent heating field from CLEOs monitor output.
    ds["mask_latent_heating"] = (
        ds["latent_heating_radius_bins"] > ds["latent_heating_radius_bins"].quantile(0.05, "time")
    ) & (ds["latent_heating_radius_bins"] < ds["latent_heating_radius_bins"].quantile(0.95, "time"))
    ds["mask_latent_heating"].attrs[
        "description"
    ] = "In order to mask out outliers in the latent heating field, the total latent heating at a timestep for a gridbox should not exceed the minimum or maximum value of the latent heating field from CLEOs monitor output."

    ds["latent_heating_radius_bins_masked"] = ds["latent_heating_radius_bins"].where(
        ds["mask_latent_heating"]
    )

    ds["latent_heating_radius_bins_time_median"] = (
        ds["latent_heating_radius_bins"]
        .sel(time=time_slice)
        .median("time", keep_attrs=True, skipna=True)
    )
    ds["latent_heating_radius_bins_time_median"].attrs["units"] = "$W m^{-3}$"
    ds["latent_heating_radius_bins_time_median"].attrs["long_name"] = "Latent heating"
    ds["latent_heating_radius_bins_time_median"].attrs[
        "description"
    ] = "Median latent heating for all timesteps"


for mp in data_dict:
    ds = data_dict[mp]["dataset"]
    add_variables(ds)

## Identify outliers


It seems the outlier is cloud ``296``

In [ ]:
null_microphysics: xr.Dataset = data_dict["null_microphysics"]["dataset"]
condensation: xr.Dataset = data_dict["condensation"]["dataset"]
collision_condensation: xr.Dataset = data_dict["collision_condensation"]["dataset"]
coalbure_condensation_cke: xr.Dataset = data_dict["coalbure_condensation_cke"]["dataset"]
coalbure_condensation_large: xr.Dataset = data_dict["coalbure_condensation_large"]["dataset"]
coalbure_condensation_small: xr.Dataset = data_dict["coalbure_condensation_small"]["dataset"]

# fig, axs = plt.subplots(ncols=5, figsize=(15, 5))
# null_microphysics["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[0])
# condensation["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[1])
# collision_condensation["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[2])
# coalbure_condensation_large["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[3])
# coalbure_condensation_small["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[4])

# remove outlier cloud_id 296
cloud_id_selection = intersect_cloud_ids[intersect_cloud_ids != 296]

null_microphysics = null_microphysics.sel(cloud_id=cloud_id_selection)
condensation = condensation.sel(cloud_id=cloud_id_selection)
collision_condensation = collision_condensation.sel(cloud_id=cloud_id_selection)
coalbure_condensation_cke = coalbure_condensation_cke.sel(cloud_id=cloud_id_selection)
coalbure_condensation_large = coalbure_condensation_large.sel(cloud_id=cloud_id_selection)
coalbure_condensation_small = coalbure_condensation_small.sel(cloud_id=cloud_id_selection)

# Input Thermodynamic profiles and PSD

``NOTE``

We can state that the **median** of the lagrangian method gives reasonable results compared to tghe direct output from the evaporation monitor of CLEO.

### Thermodynamic profiles

In [ ]:
# assert that all relative humidities are the same for all microphysics
# use the combination of all datasets to check this
import itertools

mps = data_dict.keys()
all_combinations = list(itertools.combinations(mps, r=2))

for mp1, mp2 in all_combinations:
    print(mp1, mp2)
    ds1 = data_dict[mp1]["dataset"].sel(cloud_id=cloud_id_selection)
    ds2 = data_dict[mp2]["dataset"].sel(cloud_id=cloud_id_selection)
    xr.testing.assert_equal(ds1["relative_humidity"], ds2["relative_humidity"])

null_microphysics condensation
null_microphysics collision_condensation
null_microphysics coalbure_condensation_cke
null_microphysics coalbure_condensation_large
null_microphysics coalbure_condensation_small
condensation collision_condensation
condensation coalbure_condensation_cke
condensation coalbure_condensation_large
condensation coalbure_condensation_small
collision_condensation coalbure_condensation_cke
collision_condensation coalbure_condensation_large
collision_condensation coalbure_condensation_small
coalbure_condensation_cke coalbure_condensation_large
coalbure_condensation_cke coalbure_condensation_small
coalbure_condensation_large coalbure_condensation_small


In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(
    null_microphysics["relative_humidity"].T,
    null_microphysics["gridbox_coord3"].T,
    alpha=0.6,
    # color = "k"
)
ax.set_title("Relative humidity profiles of all clouds")
ax.set_xlabel(label_from_attrs(condensation["relative_humidity"]))
ax.set_ylabel("Height [$m$]")

fig.savefig(fig_dir / "input_data" / f"relative_humidity_profiles.svg")

#### PSD in cloud layer and near cloud base

In [ ]:
data = null_microphysics
psd = data["xi_per_volume"].sel(gridbox=data["max_gridbox"]).sel(time=time_slice).mean("time")
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(
    psd["radius_bins"].expand_dims(cloud_id=psd["cloud_id"]).T,
    psd.T,
    alpha=0.6,
    # color = "k"
)
ax.set_xscale("log")
ax.set_yscale("symlog", linthresh=1e-3, linscale=0.2)
ax.set_title("Droplet Size Distribution in cloud layer\nAfter one simulation timestep")
ax.set_ylabel(label_from_attrs(data["xi_per_volume"]))
ax.set_xlabel(label_from_attrs(data["radius_bins"]))
ax.set_xlim(7e-3, 1.01e3)
ax.set_ylim(0, 5e7)

fig.savefig(fig_dir / "input_data" / f"multiplicity_profiles_max.svg")

In [ ]:
data = condensation
psd = data["xi_per_volume"].sel(gridbox=data["max_gridbox"]).sel(time=time_slice).mean("time")
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(
    psd["radius_bins"].expand_dims(cloud_id=psd["cloud_id"]).T,
    psd.T,
    alpha=0.6,
    # color = "k"
)
ax.set_xscale("log")
ax.set_yscale("symlog", linthresh=1e-3, linscale=0.2)
ax.set_title("Droplet Size Distribution in cloud layer\nAfter one simulation timestep")
ax.set_ylabel(label_from_attrs(data["xi_per_volume"]))
ax.set_xlabel(label_from_attrs(data["radius_bins"]))
ax.set_xlim(7e-3, 1.01e3)
ax.set_ylim(0, 5e7)

fig.savefig(fig_dir / "input_data" / f"multiplicity_profiles_max.svg")

In [ ]:
data = coalbure_condensation_large
psd = (
    (data["xi_per_volume"].sel(gridbox=data["max_gridbox"]) - data["xi_per_volume"].sel(gridbox=0))
    .sel(time=slice(2000, 3000))
    .mean("time")
)
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(
    psd["radius_bins"].expand_dims(cloud_id=psd["cloud_id"]).T,
    psd.T,
    alpha=0.6,
    linestyle="None",
    marker=".",
    # color = "k"
)
ax.set_xscale("log")
ax.set_yscale("symlog", linthresh=1e-3, linscale=0.2)
ax.set_title("Droplet Size Distribution in cloud layer\nAfter one simulation timestep")
ax.set_ylabel(label_from_attrs(data["xi_per_volume"]))
ax.set_xlabel(label_from_attrs(data["radius_bins"]))
ax.set_xlim(7e-3, 3.01e3)
ax.set_ylim(0, 5e7)

fig.savefig(fig_dir / "input_data" / f"multiplicity_profiles_max.svg")

In [ ]:
msd = (
    condensation["mass_represented_per_volume"]
    .sel(gridbox=condensation["max_gridbox"])
    .sel(time=time_slice)
    .mean("time")
)
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(
    msd["radius_bins"].expand_dims(cloud_id=psd["cloud_id"]).T,
    msd.T,
    alpha=0.6,
    # color = "k"
)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_title("Mass Distribution in cloud layer\nAfter one simulation timestep")
ax.set_xlabel(label_from_attrs(condensation["xi_per_volume"]))
ax.set_ylabel("Height [$m$]")

fig.savefig(fig_dir / "input_data" / f"mass_distribution_max.svg")

# Precipitation

Non smoothed precipitation shows, that individual SDs are responsible for the surface precipiation.

Smoothing the timeseries gives chance to get a more robust value of the mean rain rate.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4.5))

surface_precipitation = (
    condensation["precipitation"].sum(dim="radius_bins").chunk(dict(cloud_id=-1)).compute()
)

ax.fill_between(
    surface_precipitation["time"],
    surface_precipitation.quantile(0.10, dim="cloud_id"),
    surface_precipitation.quantile(0.90, dim="cloud_id"),
    color=[0.9, 0.9, 0.9],
    alpha=1,
    label="10% - 90%",
)
ax.fill_between(
    surface_precipitation["time"],
    surface_precipitation.quantile(0.25, dim="cloud_id"),
    surface_precipitation.quantile(0.75, dim="cloud_id"),
    color=[0.75, 0.75, 0.75],
    alpha=1,
    label="25% - 75%",
)


ax.plot(
    surface_precipitation["time"],
    surface_precipitation.median("cloud_id"),
    color="k",
    linestyle="-",
    alpha=1,
    label="median",
)


for cid in clouds_dict:
    cloud_id = clouds_dict[cid]["cloud_id"]
    color = clouds_dict[cid]["color"]
    ax.plot(
        surface_precipitation["time"],
        surface_precipitation.sel(cloud_id=cloud_id),
        color=color,
        linestyle="-",
        alpha=0.5,
        label=f"cloud {cloud_id}",
    )

ax.set_ylabel(r"Surface precipitation $mm/h$")
ax.set_xlabel(r"Time $s$")

ax.legend(loc="upper left")

fig.savefig(fig_dir / "surface_precipitation_unsmoothed.svg")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4.5))

mean_color = mcolors.CSS4_COLORS["darkorange"]

# surface_precipitation = (
#     condensation["precipitation"]
#     .sum(dim="radius_bins")
#     .chunk(dict(cloud_id=-1))
#     .rolling(time=50, center=True)
#     .mean()
#     .compute()
# )

data = condensation.sel(time=time_slice)

surface_prec = data["precipitation"].sum(dim="radius_bins").chunk(dict(cloud_id=-1))

surface_prec_cloud_mean, surface_prec_cloud_sem = mean_and_stderror_of_mean(
    data=surface_prec,
    dims=("cloud_id",),
)

surface_prec_smooth = surface_prec.rolling(time=50, center=True).mean()
surface_prec_cloud_mean_smooth = surface_prec_cloud_mean.rolling(time=50, center=True).mean()
surface_prec_cloud_sem_smooth = surface_prec_cloud_sem.rolling(time=50, center=True).mean()

print("old dims", data["precipitation"].attrs["units"])
print("new dim", data["precipitation"].attrs["units"])
surface_prec.attrs["units"] = data["precipitation"].attrs["units"]
surface_prec.attrs["long_name"] = data["precipitation"].attrs["long_name"]


ax.plot(
    surface_prec_smooth["time"],
    surface_prec_smooth.T,
    color="grey",  # [0.75, 0.75, 0.75],
    linestyle="-",
    alpha=0.5,
    zorder=0,
    # label="Mean",
)

ax.fill_between(
    surface_prec_cloud_mean_smooth["time"],
    surface_prec_cloud_mean_smooth - 2 * surface_prec_cloud_sem_smooth,
    surface_prec_cloud_mean_smooth + 2 * surface_prec_cloud_sem_smooth,
    color=adjust_lightness_array([mean_color], 1.5),
    alpha=0.7,
    label="2 SEM",
    zorder=1,
)
ax.fill_between(
    surface_prec_cloud_mean_smooth["time"],
    surface_prec_cloud_mean_smooth - surface_prec_cloud_sem_smooth,
    surface_prec_cloud_mean_smooth + surface_prec_cloud_sem_smooth,
    color=adjust_lightness_array([mean_color], 1.2),
    alpha=0.7,
    label="1 SEM",
    zorder=2,
)

ax.plot(
    surface_prec_cloud_mean_smooth["time"],
    surface_prec_cloud_mean_smooth,
    color="darkorange",
    linestyle="-",
    alpha=1,
    label="Mean",
    zorder=3,
)


for cid in clouds_dict:
    cloud_id = clouds_dict[cid]["cloud_id"]
    color = clouds_dict[cid]["color"]
    ax.plot(
        surface_prec_smooth["time"],
        surface_prec_smooth.sel(cloud_id=cloud_id),
        color=color,
        linestyle="-",
        alpha=0.5,
        label=f"cloud {cloud_id}",
    )

ax.set_ylabel(r"Surface precipitation $mm/h$")
ax.set_xlabel(r"Time $s$")

ax.legend(loc="upper left")

# fig.savefig(fig_dir / "surface_precipitation_smoothed.svg")

old dims mm/h
new dim mm/h


#### Histogram of Precipiation and its relation to the mass in the surface gridbox

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4.5))

ax.hist(
    surface_precipitation.sel(time=time_slice).mean("time"),
    bins=20,
    color="k",
    alpha=0.5,
    density=True,
    label="Histogram",
)
ax.axvline(
    surface_precipitation.sel(time=time_slice).mean("time").median("cloud_id"),
    color="k",
    linestyle="-",
    alpha=1,
    label=f"Median",
)
for cid in clouds_dict:
    cloud_id = clouds_dict[cid]["cloud_id"]
    color = clouds_dict[cid]["color"]
    ax.axvline(
        surface_precipitation.sel(cloud_id=cloud_id).sel(time=time_slice).mean("time"),
        color=color,
        linestyle="-",
        alpha=0.5,
        label=f"cloud {cloud_id}",
    )

ax.set_xlabel(r"Mean surface precipitation $mm/h$")
ax.set_ylabel("Density")
ax.set_title("Histogram of mean surface precipitation")

fig.savefig(fig_dir / "histogram_surface_precipitation.svg")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4.5))

ax.hist(
    24 * surface_precipitation.sel(time=time_slice).mean("time"),
    bins=20,
    color="k",
    alpha=0.5,
    density=True,
    label="Histogram",
)
ax.axvline(
    24 * surface_precipitation.sel(time=time_slice).mean("time").median("cloud_id"),
    color="k",
    linestyle="-",
    alpha=1,
    label=f"Median",
)
for cid in clouds_dict:
    cloud_id = clouds_dict[cid]["cloud_id"]
    color = clouds_dict[cid]["color"]
    ax.axvline(
        24 * surface_precipitation.sel(cloud_id=cloud_id).sel(time=time_slice).mean("time"),
        color=color,
        linestyle="-",
        alpha=0.5,
        label=f"cloud {cloud_id}",
    )

ax.set_xlabel(r"Mean surface precipitation $mm/d$")
ax.set_ylabel("Density")
ax.set_title("Histogram of mean surface precipitation")

fig.savefig(fig_dir / "histogram_surface_precipitation_day.svg")

In [ ]:
data = condensation.sel(time=time_slice)

surface_prec = data["precipitation"].sum(dim="radius_bins").chunk(dict(cloud_id=-1))

print("old dims", data["precipitation"].attrs["units"])
print("new dim", data["precipitation"].attrs["units"])
surface_prec.attrs["units"] = data["precipitation"].attrs["units"]
surface_prec.attrs["long_name"] = data["precipitation"].attrs["long_name"]


surface_prec_time_mean, surface_prec_time_sem = mean_and_stderror_of_mean(
    data=surface_prec,
    dims=("time",),
)
surface_prec_time_cloud_mean, surface_prec_time_cloud_sem = mean_and_stderror_of_mean(
    data=surface_prec_time_mean,
    dims=("cloud_id",),
    data_std=surface_prec_time_sem,
)


fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(
    surface_prec_time_mean,
    bins=np.arange(0, 0.22, 0.01),
    color="k",
    alpha=0.5,
    density=True,
)
ax.axvline(
    surface_prec_time_cloud_mean,
    color="darkorange",
    label="Mean",
)

ax.fill_betweenx(
    [0, 16],
    surface_prec_time_cloud_mean - 2 * surface_prec_time_cloud_sem,
    surface_prec_time_cloud_mean + 2 * surface_prec_time_cloud_sem,
    color="orange",
    alpha=0.3,
    label="2 SEM",
)

# ax.axvline(
#     lh_sum_time_mean.compute().median("cloud_id"),
#     color="red",
#     label="Median",
#     linestyle = "--"
# )


ax.set_xlabel(label_from_attrs(surface_prec))
ax.set_ylabel("Density")
ax.set_title(f"Histogram of {label_from_attrs(surface_prec, return_units=False)}")

# ax.set_ylim(0, 0.065)

old dims mm/h
new dim mm/h


Text(0.5, 1.0, 'Histogram of Precipitation')

In [ ]:
plt.hist(surface_prec_time_mean, density=True)

(array([ 7.01203813,  1.5582307 ,  2.33734604,  7.01203813,  4.67469209,
        12.46584557,  9.34938418,  3.11646139,  2.33734604,  1.5582307 ]),
 array([0.0012471 , 0.02069418, 0.04014125, 0.05958833, 0.07903541,
        0.09848248, 0.11792956, 0.13737663, 0.15682371, 0.17627078,
        0.19571786]),
 <BarContainer object of 10 artists>)

### Context to rain literature

The rain rates are siginificantly lower than “rain intensities of 0.9 mm h−1” ([Vogel et al., 2021, p. 16625](zotero://select/library/items/TNDVW8WC)) ([pdf](zotero://open-pdf/library/items/FZ2HISRL?page=17&annotation=PYMP44CU))

This might be due to the fact, that we did not sample through any cold pool or strong precipiating clouds.
Also take a look at the different seasons.


On the other hand Sakat et al showed very wide rang of rain rates “mean rain rates and RWC are between 0.01 - 31 mm d−1” ([Sarkar et al., 2023, p. 12685](zotero://select/library/items/G2B2A8IK)) ([pdf](zotero://open-pdf/library/items/ZNPPEKFT?page=15&annotation=FVB8GLGJ))

In [ ]:
fig, axs = plt.subplots(figsize=(10, 5), ncols=3, sharex=True, sharey=True)
axs[0].scatter(
    surface_precipitation.sel(time=time_slice).mean("time"),
    condensation["mass_represented"]
    .sel(gridbox=condensation["max_gridbox"])
    .sel(time=time_slice)
    .sum("radius_bins")
    .mean("time"),
    marker=".",
    # label=f"cloud {cloud_id.values}",
)
axs[0].set_xlabel(r"Surface precipitation $mm/h$")
axs[0].set_ylabel("LWC [g m$^{-3}$]")
axs[0].set_title("Cloud layer")

axs[1].scatter(
    surface_precipitation.sel(time=time_slice).mean("time"),
    condensation["mass_represented"]
    .sel(gridbox=condensation["max_gridbox"] - 1)
    .sel(time=time_slice)
    .sum("radius_bins")
    .mean("time"),
    marker=".",
    # label=f"cloud {cloud_id.values}",
)
axs[1].set_xlabel(r"Surface precipitation $mm/h$")
axs[1].set_ylabel("Mass [g m$^{-3}$]")
axs[1].set_title("Top sub cloud layer")

axs[2].scatter(
    surface_precipitation.sel(time=time_slice).mean("time"),
    condensation["mass_represented"].sel(gridbox=0).sel(time=time_slice).sum("radius_bins").mean("time"),
    marker=".",
    # label=f"cloud {cloud_id.values}",
)
axs[2].set_xlabel(r"Surface precipitation $mm/h$")
axs[2].set_ylabel("Mass [g m$^{-3}$]")
axs[2].set_title("Surface gridbox")

fig.savefig(fig_dir / "scatter_surface_precipitation_LWC.svg")